In [1]:
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF/'
timestep_nr = 0
var = 'e'
dimension = 'z'

In [2]:
import xarray as xr
import numpy as np
import zarr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from utils import visualization_tools

In [3]:
ncar_data = xr.open_dataset(raw_ncar_folder_path + "jhd." + str(timestep_nr).zfill(3) + ".nc")

In [4]:
# ncar_data.info

In [ ]:
visualization_tools.animate_cube(ncar_data[var], dimension=dimension)